In [2]:
import math 
import torch
import random
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from highlight import *
from xentlang import X

In [5]:
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

model.save_pretrained("models/gpt2-M0")
tokenizer.save_pretrained("models/gpt2-M0")

('models/gpt2-M0/tokenizer_config.json',
 'models/gpt2-M0/special_tokens_map.json',
 'models/gpt2-M0/vocab.json',
 'models/gpt2-M0/merges.txt',
 'models/gpt2-M0/added_tokens.json',
 'models/gpt2-M0/tokenizer.json')

In [6]:
model = AutoModelForCausalLM.from_pretrained("models/gpt2-M0/", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("models/gpt2-M0/", local_files_only=True)

In [ ]:
def generate_data():
    task_manager = HilightWiki()
    textok, t1, t2 = task_manager.extract()

    min_y_window = int(sub_window_size)
    i = random.randint(0, textok.size(-1) - min_y_window)
    y = textok[0][i:i+min_y_window]

    hilisorted = task_manager.xent_sort(y.unsqueeze(0), t1, t2)

    response = detokenize_l(hilisorted)

    keep_best = 20

    red_best = X().comma.join(response[:keep_best])
    blue_best = X().comma.join(response[-keep_best:])

    min_y_window = int(sub_window_size)

    i = random.randint(0, textok.size(-1) - min_y_window)
    slice = textok[0][i:i+min_y_window]
    ywidth = 3
    o = slice[:ywidth]
    c = slice[-ywidth:]

    y = (detokenize_w(o), detokenize_w(c))
    # print(y)
    x1 = detokenize_0(t1)
    x2 = detokenize_0(t2)

    text = detokenize_0(textok)
    func = redblue(y, x1, x2)

    output = text+"\n"+func+"\n"+red_best+X().comma+blue_best

    return output

In [ ]:
import pyinstrument

with pyinstrument.profile():
    text = generate_data()


In [ ]:
# import pstats
# from pstats import SortKey
# import cProfile
# cProfile.run("generate_data()", "restats")
# p = pstats.Stats("restats")
# p.strip_dirs().sort_stats(SortKey.TIME).print_stats()

In [ ]:
# gen = 1e5
# separator = "£££$$$£££"

# for i in tqdm(range(int(gen))):
#     output = generate_data()
#     with open("highlighting_data.txt", "a") as f:
#         f.write(output + separator)